# Dados e Aprendizagem Automática
### Imports


In [ ]:
try:
    import sklearn
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np
    import time
    import shap
    print("Todas as bibliotecas estão instaladas!")
except ImportError as e:
    print(f"Erro: {e}")

In [2]:
pd.set_option("display.max_columns", None) #desta maneira conseguimos com que apareçam todas as colunas, tirando a limitação de X colunas

Load do datasets|
----------

In [3]:
train = pd.read_csv("datasets/train_radiomics_hipocamp.csv")
test = pd.read_csv("datasets/test_radiomics_hipocamp.csv")
controlo = pd.read_csv("datasets/train_radiomics_occipital_CONTROL.csv")

Concatenação dos datasets de treino e teste, de forma a facilitar o tratamento de dados |
----------

In [4]:
concat =  pd.concat([train, test], ignore_index=True)

Comecemos por transformar o atributo Transition em 0/1/2/3/4 para facilitar o seu uso( e melhora os resultados?) |
----------

In [5]:
target_mapping = {'AD-AD': 0, 'CN-CN': 1, 'MCI-AD': 2, 'MCI-MCI': 3, 'CN-MCI': 4}
if 'Transition' in concat.columns:
    concat['Transition'] = concat['Transition'].map(target_mapping)

Remover colunas desnecessárias |
----------


In [6]:
cols_to_drop = [col for col in concat.columns if col.startswith('diagnostics')] #informação imagem(versões usadas,etc)
cols_to_drop.extend(['ID', 'Mask','Image'])# id, caminhos de imagem e mask
concat = concat.drop(columns=cols_to_drop)

Remover as colunas onde o valor não varia nas linhas |
--------

In [ ]:
columns_single_value = concat.columns[concat.nunique() == 1]
print(columns_single_value)
concat = concat.drop(columns=columns_single_value)

Remover colunas com variação baixa |
--------


In [ ]:
concat_without_transition = concat.drop(columns=['Transition'])
std_values = concat_without_transition.std()
std_ordered = std_values.sort_values().index
std_list = [f"{ss} ----> {std_values[ss]}" for ss in std_ordered]
print("\n".join(std_list[:20]))

In [ ]:
features_to_drop = []
for ss in std_ordered:
    #print(f"{feature}: {importance}")

    if std_values[ss] < 0.01:
        features_to_drop.append(ss)

print(len(features_to_drop))

concat = concat.drop(columns=features_to_drop)

concat.shape

Separar datasets |
--------

In [10]:
train_dataset = concat[concat['Transition'].notna()]
test_dataset = concat[concat['Transition'].isna()]
test_dataset = test_dataset.drop(columns=['Transition'])

Verificar se foram bem separados |
------------

Primeiro, verificar se há valores null em Transition

In [ ]:
print("Valores nulos:", train_dataset['Transition'].isna().sum())
if 'Transition' in test_dataset.columns:
    print("Errado em:", test_dataset['Transition'].notna().sum())
else:
    print("Correto.")


Vamos verificar o tamanho dos conjuntos

In [ ]:
print("Número de linhas no dataset concatenado:", len(concat))
print("Número de linhas no dataset de treino:", len(train_dataset))
print("Número de linhas no dataset de teste:", len(test_dataset))
print("Soma das linhas de treino e teste:", len(train_dataset) + len(test_dataset))

Datasets foram devidamente tratados e separados ! |
------------

Guardar datasets |
-------

In [13]:
test_dataset.to_csv('datasets/test_clean.csv', index=False)
train_dataset.to_csv('datasets/train_clean.csv', index=False)

Tratamento done |
-----